# Objective Evaluations - Scores

In [ ]:
import pandas as pd
from scipy.stats import f_oneway

In [ ]:
resp_data = pd.read_csv('questionnaire\\long_term\\response_summary_long.csv')
resp_data.head(5)

## Mix-up

In [ ]:
# Group by 'method' and compute mean and std of 'score'
grouped_data = resp_data.groupby('method')['score'].agg(['mean', 'std'])

# Perform ANOVA test
anova_result = f_oneway(*[group['score'] for name, group in resp_data.groupby('method')])

# Print the mean and std of 'score' for each 'method'
print("Mean and Standard Deviation of 'score' by 'method':")
print(grouped_data)

# Print the ANOVA test result
print("\nANOVA Test Result:")
print("F-value:", anova_result.statistic)
print("p-value:", anova_result.pvalue)

## Add strategy

In [ ]:
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x='strategy', y='score', hue='method', data=resp_data, palette='viridis')

# Set plot title and labels
plt.title('Scores by Strategy and Method')
plt.xlabel('Strategy')
plt.ylabel('Score')

# Display the plot
plt.show()

## Shirley's Method

for each participant {
    baseline_score_g = mean(method==0, G);
    baseline_score_o = mean(method==0, O);
    for each passage whose method == 1 {
        do pair t-test between the score (compare the score and the corresponding baseline, decided by G or O)
    }
}

## Add overall recall and theme recall

recall: overall recall

theme: the question 6

overall: the total score

In [ ]:
import pandas as pd
from scipy.stats import ttest_rel

# Step 1: Load the data and split by method
df = pd.read_csv('/Users/fukexue/film_learning_study3/final score with all data.csv')

# Split into two dataframes based on method
df_method_0 = df[df['method'] == 0]
df_method_1 = df[df['method'] == 1]

# Function to compute baseline scores
def compute_baseline(df_method_0, column_name):
    baseline_scores = df_method_0.groupby(['participant', 'corpus'])[column_name].mean().reset_index()
    baseline_scores.rename(columns={column_name: f'baseline_{column_name}'}, inplace=True)
    return baseline_scores

# Compute baseline scores for 'score', 'recall', and 'theme'
baseline_withcues = compute_baseline(df_method_0, 'withcues')
baseline_withoutcues = compute_baseline(df_method_0, 'withoutcues')


# Merge the baseline scores with method==1 dataframe
df_method_1_merged = pd.merge(df_method_1, baseline_withcues, on=['participant', 'corpus'])
df_method_1_merged = pd.merge(df_method_1_merged, baseline_withoutcues, on=['participant', 'corpus'])


# Function to perform t-test and print results
def perform_t_test(df_method_1_merged, column_name):
    t_stat, p_value = ttest_rel(df_method_1_merged[column_name], df_method_1_merged[f'baseline_{column_name}'])
    average_score_method_0 = df_method_0[column_name].mean()
    average_score_method_1 = df_method_1_merged[column_name].mean()
    print(f"Average {column_name} for method 0:", average_score_method_0)
    print(f"Average {column_name} for method 1:", average_score_method_1)
    print(f"T-statistic for {column_name}:", t_stat)
    print(f"P-value for {column_name}:", p_value)
    print('\n')

# Perform t-tests for 'score', 'recall', and 'theme'
for column in ['withcues', 'withoutcues']:
    perform_t_test(df_method_1_merged, column)

# Save the merged dataframe with baseline columns to a single CSV file
df_method_1_merged.to_csv('questionnaire/long_term/t-test_results.csv', index=False)



# Initialize a list to store the results
results = []

# Function to perform t-test and store results
def perform_t_test_by_corpus(df_method_1_merged, column_name):
    grouped = df_method_1_merged.groupby('corpus')
    for corpus_name, group in grouped:
        t_stat, p_value = ttest_rel(group[column_name], group[f'baseline_{column_name}'])
        average_score_method_0 = df_method_0[df_method_0['corpus'] == corpus_name][column_name].mean()
        average_score_method_1 = group[column_name].mean()
        
        # Store the results in the list
        results.append({
            'corpus': corpus_name,
            'metric': column_name,
            'average_method_0': average_score_method_0,
            'average_method_1': average_score_method_1,
            't_statistic': t_stat,
            'p_value': p_value
        })

# Perform t-tests for 'score', 'recall', 'theme', and 'overall' grouped by corpus
for column in ['withcues', ' withoutcues']:
    perform_t_test_by_corpus(df_method_1_merged, column)

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Output the results DataFrame
print(results_df)


## Visualization and P value calculation for each strategy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import json

# Load the uploaded CSV data
file_path = 'questionnaire/long_term/response_summary_long.csv'
data = pd.read_csv(file_path)

# Extract unique strategies
strategies = data['strategy'].unique()

# Initialize dictionary to store results
results = {}

# Iterate over each strategy and column of interest
for strategy in strategies:
    strategy_data = data[data['strategy'] == strategy]
    results[strategy] = {}
    
    for column in ['score', 'recall', 'theme', 'overall']:
        results[strategy][column] = {}
        
        # Prepare data for ANOVA
        method_groups = [strategy_data[strategy_data['method'] == method][column].values for method in strategy_data['method'].unique()]
        
        # Perform one-way ANOVA
        f_stat, p_value = stats.f_oneway(*method_groups)
        
        # Store results in dictionary
        results[strategy][column] = {
            'f_stat': f_stat,
            'p_value': p_value
        }
        
        # Plotting for each column
        fig, ax = plt.subplots(figsize=(12, 6))
        boxplot_data = method_groups
        labels = [f'Method {method}' for method in strategy_data['method'].unique()]
        
        ax.boxplot(boxplot_data, labels=labels, patch_artist=True)
        plt.xlabel('Method')
        plt.ylabel(column.capitalize())
        plt.title(f'{column.capitalize()} Comparison within {strategy} (ANOVA)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

# Optionally, save the results to a JSON file
output_file = '/Users/fukexue/film_learning_study3/questionnaire/long_term/strategy_comparison_anova_results.json'
with open(output_file, 'w') as json_file:
    json.dump(results, json_file, indent=4)

# Print the results dictionary for review
print(json.dumps(results, indent=4))


## Final data： To test repeated measure anova with strategy 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.anova import AnovaRM
import json

# Load the uploaded CSV data
file_path = 'final score with all data.csv'
data = pd.read_csv(file_path)

# Extract unique strategies
strategies = data['strategy'].unique()

# Initialize dictionary to store ANOVA results and means of method 0 and method 1
results = {}

# Prepare subplots for combined visualization
fig, axs = plt.subplots(2, 1, figsize=(16, 12))
columns = ['withoutcues', 'withcues']
axes = axs.flatten()

# Iterate over each column of interest
for i, column in enumerate(columns):
    results[column] = {}
    
    # Prepare data for plotting and analysis
    combined_data = []
    labels = []

    for strategy in strategies:
        strategy_data = data[data['strategy'] == strategy]
        
        # Reformat the data for ANOVA
        long_format_data = pd.melt(strategy_data, id_vars=['participant', 'strategy', 'method'], value_vars=[column],
                                  var_name='variable', value_name='value')
        
        # Perform Repeated Measures ANOVA
        model = AnovaRM(long_format_data, 'value', 'participant', within=['method'])
        results[column][strategy] = model.fit()
        
        # Prepare data for boxplot
        combined_data.append(strategy_data[strategy_data['method'] == 0][column].values)
        combined_data.append(strategy_data[strategy_data['method'] == 1][column].values)
        labels.append(f'{strategy} (Method 0)')
        labels.append(f'{strategy} (Method 1)')
    
    # Plotting combined boxplot for each column
    axes[i].boxplot(combined_data, labels=labels, patch_artist=True)
    axes[i].set_title(f'{column.capitalize()} across all Strategies')
    axes[i].set_xlabel('Strategy and Method')
    axes[i].set_ylabel(column.capitalize())
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Display the combined results dictionary with means and ANOVA results
anova_results = {}
for column, strategy_results in results.items():
    anova_results[column] = {}
    for strategy, result in strategy_results.items():
        anova_results[column][strategy] = {
            'anova_table': result.summary().as_text()
        }

print(json.dumps(anova_results, indent=4))


# final data To test repeated measure(one way) anova on each material 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import json

# Load the uploaded CSV data
file_path = 'final score with all data.csv'
data = pd.read_csv(file_path)

# Extract unique topics
topics = data['topic'].unique()

# Initialize dictionary to store ANOVA results and p-values
results = {}

# Prepare subplots for combined visualization
fig, axs = plt.subplots(2, 1, figsize=(18, 12))
columns = ['withoutcues', 'withcues']
axes = axs.flatten()

# Iterate over each dimension (column)
for i, column in enumerate(columns):
    all_combined_data = []
    all_labels = []
    
    # Initialize results for this column
    if column not in results:
        results[column] = {}
    
    for topic in topics:
        topic_data = data[data['topic'] == topic]
        
        # Perform One-Way ANOVA without considering participant
        try:
            model = sm.formula.ols(f'{column} ~ C(method)', data=topic_data).fit()
            anova_table = sm.stats.anova_lm(model, typ=2)
            p_value = anova_table['PR(>F)']['C(method)']
            results[column][topic] = {
                'anova_table': anova_table.to_string(),
                'p_value': p_value
            }
        except Exception as e:
            results[column][topic] = {
                'anova_table': str(e),
                'p_value': 'nan'
            }

        # Prepare data for plotting
        method_0_data = topic_data[topic_data['method'] == 0][column].values
        method_1_data = topic_data[topic_data['method'] == 1][column].values
        
        all_combined_data.append(method_0_data)
        all_combined_data.append(method_1_data)
        all_labels.append(f'{topic} Method 0')
        all_labels.append(f'{topic} Method 1')
    
    # Plotting combined boxplot for each column
    axes[i].boxplot(all_combined_data, labels=all_labels, patch_artist=True)
    axes[i].set_title(f'{column.capitalize()} Across Topics')
    axes[i].set_xlabel('Topic and Method')
    axes[i].set_ylabel(column.capitalize())
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Output the results with p-values
results_json = json.dumps(results, indent=4)

# Extract and print p-values for each topic
for column in columns:
    print(f"\nP-values for '{column}':")
    if column in results:
        for topic in topics:
            p_value = results[column].get(topic, {}).get('p_value', 'nan')
            print(f"{topic}: {p_value}")

# Print the full results JSON
results_json


## Topic‘s influence 

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# 假设数据已加载
file_path = 'questionnaire/long_term/response_summary_long.csv'  # 替换为您的文件路径
data = pd.read_csv(file_path)

# 要分析的四个维度
dimensions = ['score', 'recall', 'theme', 'overall']

# 循环分析每个维度
for dimension in dimensions:
    print(f"\nAnalysis for {dimension}:")

    # 创建线性回归模型，评估 topic 的影响
    model = ols(f'{dimension} ~ topic', data=data).fit()
    
    # 生成 ANOVA 表
    anova_table = sm.stats.anova_lm(model, typ=2)
    print(f"ANOVA results for {dimension}:")
    print(anova_table)

    # 执行 Tukey HSD 测试，进一步分析 topic 的影响
    print(f"\nTukey HSD results for {dimension}:")
    tukey = pairwise_tukeyhsd(endog=data[dimension], groups=data['topic'], alpha=0.05)
    print(tukey)


# method=0 or method=1' tukey

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# 加载数据
file_path = 'questionnaire/long_term/response_summary_long.csv'  # 替换为您的文件路径
data = pd.read_csv(file_path)

# 要分析的四个维度
dimensions = ['score', 'recall', 'theme', 'overall']

# 分别针对 method=0 和 method=1 进行分析
methods = [0, 1]

for method in methods:
    print(f"\nANOVA and Tukey HSD analysis for method={method}:")

    # 过滤数据只保留当前 method 的情况
    method_data = data[data['method'] == method]
    
    for dimension in dimensions:
        print(f"\nAnalyzing {dimension} (Method = {method}):")
        
        # ANOVA 分析
        model = ols(f'{dimension} ~ C(topic)', data=method_data).fit()
        anova_table = anova_lm(model, typ=2)
        print("ANOVA results:")
        print(anova_table)
        
        # 提取p值
        p_value = anova_table["PR(>F)"][0]
        print(f"p-value: {p_value:.4f}")
        
        # 如果 ANOVA 结果显示 topic 的影响显著（p值小于0.05），进行 Tukey HSD 分析
        if p_value < 0.05:
            print(f"\nTukey HSD results for {dimension} (Method = {method}):")
            tukey = pairwise_tukeyhsd(endog=method_data[dimension], groups=method_data['topic'], alpha=0.05)
            print(tukey.summary())
        else:
            print(f"No significant effect of topic on {dimension} (Method = {method}). Skipping Tukey HSD analysis.")


## Short term results 

In [ ]:
import pandas as pd
from scipy.stats import ttest_rel

# Step 1: Load the data and split by method
df = pd.read_csv('questionnaire/short_term/response_summary.csv')  # Replace with your CSV file path

# Split into two dataframes based on method
df_method_0 = df[df['method'] == 0]
df_method_1 = df[df['method'] == 1]

# Function to compute baseline scores
def compute_baseline(df_method_0, column_name):
    baseline_scores = df_method_0.groupby(['participant', 'corpus'])[column_name].mean().reset_index()
    baseline_scores.rename(columns={column_name: f'baseline_{column_name}'}, inplace=True)
    return baseline_scores

# List of columns to perform t-tests on
columns_to_test = [
    'score', 'competence', 'activity', 'frustration', 'pleasure', 'pressure', 
    'effort', 'immersion', 'physicality', 'difficulty', 'familarity', 'understanding'
]

# Initialize a list to store the results
results = []

# Function to perform t-test and store results by corpus
def perform_t_test_by_corpus(df_method_1_merged, column_name):
    grouped = df_method_1_merged.groupby('corpus')
    for corpus_name, group in grouped:
        t_stat, p_value = ttest_rel(group[column_name], group[f'baseline_{column_name}'])
        average_score_method_0 = df_method_0[df_method_0['corpus'] == corpus_name][column_name].mean()
        average_score_method_1 = group[column_name].mean()
        
        # Store the results in the list
        results.append({
            'corpus': corpus_name,
            'metric': column_name,
            'average_method_0': average_score_method_0,
            'average_method_1': average_score_method_1,
            't_statistic': t_stat,
            'p_value': p_value
        })

# Perform t-tests for each column grouped by corpus
for column in columns_to_test:
    # Compute baseline scores for the current column
    baseline_score = compute_baseline(df_method_0, column)
    
    # Merge the baseline scores with method==1 dataframe
    df_method_1_merged = pd.merge(df_method_1, baseline_score, on=['participant', 'corpus'])
    
    # Perform t-test for the current column grouped by corpus
    perform_t_test_by_corpus(df_method_1_merged, column)

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Output the results DataFrame
print(results_df)

# Optionally, save the results to a CSV file
results_df.to_csv('questionnaire/short_term/t-test_results_all_columns.csv', index=False)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import json

# 读取CSV数据到DataFrame
data = pd.read_csv('questionnaire/short_term/t-test_results.csv')

# 提取感兴趣的列
columns_of_interest = [
   'score', 'competence', 'activity', 'frustration', 'pleasure', 'pressure', 
    'effort', 'immersion', 'physicality', 'difficulty', 'familarity', 'understanding'
]

# 提取各策略的唯一值
strategies = data['strategy'].unique()
results = {}

# 对每个感兴趣的列进行迭代
for column in columns_of_interest:
    boxplot_data = []
    labels = []
    results[column] = {}
    
    for strategy in strategies:
        strategy_data = data[data['strategy'] == strategy]
        
        # 提取基准数据和策略数据
        baseline_data = strategy_data['baseline_' + column].values
        strategy_data_values = strategy_data[column].values
        
        # 计算均值
        baseline_mean = baseline_data.mean()
        strategy_mean = strategy_data_values.mean()
        
        # 进行配对t检验以获得p值
        t_stat, p_value = stats.ttest_rel(baseline_data, strategy_data_values)
        
        # 将结果存储到字典中
        results[column][strategy] = {
            'baseline_mean': baseline_mean,
            'strategy_mean': strategy_mean,
            'p_value': p_value
        }
        
        # 为箱线图准备数据
        boxplot_data.append(baseline_data)
        boxplot_data.append(strategy_data_values)
        labels.append(f'{strategy} baseline')
        labels.append(f'{strategy} w/ strategy')
    
    # 绘图
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.boxplot(boxplot_data, labels=labels, patch_artist=True)

    plt.xlabel('Strategy')
    plt.ylabel(column.capitalize())
    plt.title(f'{column.capitalize()} of Baseline vs. Strategy')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# 可选：将结果保存到JSON文件
with open('strategy_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)

# 打印结果字典以供查看
print(json.dumps(results, indent=4))
